<a href="https://colab.research.google.com/github/Kibet-Rotich/data-science-learning/blob/master/data_manipulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧹 Data Cleaning & Transformation


## 📦 1. Setup and Data Generation

We’ll create a **synthetic dataset** that mimics messy, real-world data — including missing values, duplicates, inconsistent data types, typos, and outliers.

In [ ]:
import pandas as pd
import numpy as np
import random

np.random.seed(42)

n = 60

# Simulate some messy data
data = {
    'Customer_ID': np.random.randint(1000, 1100, n),
    'Name': [random.choice(['Alice', 'Bob', 'Catherine', 'David', 'Evelyn', 'Frank', 'Grace', 'Hannah']) for _ in range(n)],
    'Gender': [random.choice(['M', 'F', 'Male', 'Female', 'male', 'female', np.nan]) for _ in range(n)],
    'Age': [random.choice([20, 25, 30, 35, 40, 45, 50, np.nan, 'thirty']) for _ in range(n)],
    'Join_Date': [random.choice(['2021-01-05', '2020-06-10', '2019/12/15', '15-07-2022', np.nan]) for _ in range(n)],
    'City': [random.choice(['Nairobi', 'Mombasa', 'kisumu', 'NAIROBI', 'Eldoret', np.nan]) for _ in range(n)],
    'Income': [random.choice([35000, 50000, 70000, 90000, np.nan, 200000, 1000000]) for _ in range(n)],
    'Satisfaction_2022': np.random.randint(50, 100, n),
    'Satisfaction_2023': np.random.randint(40, 100, n),
    'Preferred_Channel': [random.choice(['Online', 'In-store', 'Both', np.nan]) for _ in range(n)]
}

# Add duplicates deliberately
df = pd.DataFrame(data)
df = pd.concat([df, df.iloc[:5]], ignore_index=True)

df.head(10)

,Customer_ID,Name,Gender,Age,Join_Date,City,Income,Satisfaction_2022,Satisfaction_2023,Preferred_Channel
0,1051,Alice,Female,thirty,15-07-2022,kisumu,90000.0,77,98,Both
1,1092,Grace,Male,40,2020-06-10,NaN,50000.0,96,44,In-store
2,1014,Catherine,M,35,2019/12/15,Mombasa,90000.0,56,81,In-store
3,1071,Frank,NaN,20,2020-06-10,NAIROBI,70000.0,93,78,Online
4,1060,David,Female,45,2020-06-10,Nairobi,35000.0,57,97,NaN
5,1020,Evelyn,F,25,15-07-2022,Eldoret,90000.0,96,80,Online
6,1082,Hannah,NaN,50,2021-01-05,kisumu,NaN,84,67,In-store
7,1086,Catherine,Male,25,2020-06-10,kisumu,90000.0,63,46,NaN
8,1074,Evelyn,Female,25,2021-01-05,Nairobi,90000.0,66,48,Both
9,1074,Hannah,Female,NaN,15-07-2022,Eldoret,1000000.0,85,47,Online


## 🔍 2. Initial Data Profiling

Before cleaning, let’s inspect the data structure, types, and common problems.

In [ ]:
df.info()
print('\nMissing values per column:\n', df.isna().sum())
print('\nUnique values per column:\n', df.nunique())
print('\nSummary statistics:')
display(df.describe(include='all'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Customer_ID        65 non-null     int64  
 1   Name               65 non-null     object 
 2   Gender             57 non-null     object 
 3   Age                60 non-null     object 
 4   Join_Date          54 non-null     object 
 5   City               57 non-null     object 
 6   Income             58 non-null     float64
 7   Satisfaction_2022  65 non-null     int64  
 8   Satisfaction_2023  65 non-null     int64  
 9   Preferred_Channel  51 non-null     object 
dtypes: float64(1), int64(3), object(6)
memory usage: 5.2+ KB

Missing values per column:
 Customer_ID           0
Name                  0
Gender                8
Age                   5
Join_Date            11
City                  8
Income                7
Satisfaction_2022     0
Satisfaction_2023     0
Preferred_Chan

,Customer_ID,Name,Gender,Age,Join_Date,City,Income,Satisfaction_2022,Satisfaction_2023,Preferred_Channel
count,65.000000,65,57,60.0,54,57,58.000000,65.000000,65.000000,51
unique,NaN,8,6,8.0,4,5,NaN,NaN,NaN,3
top,NaN,Alice,male,35.0,2020-06-10,NAIROBI,NaN,NaN,NaN,Online
freq,NaN,11,16,10.0,19,15,NaN,NaN,NaN,21
mean,1049.892308,NaN,NaN,NaN,NaN,NaN,224224.137931,74.138462,69.892308,NaN
std,29.418384,NaN,NaN,NaN,NaN,NaN,317837.818243,15.007162,16.815286,NaN
min,1001.000000,NaN,NaN,NaN,NaN,NaN,35000.000000,50.000000,40.000000,NaN
25%,1021.000000,NaN,NaN,NaN,NaN,NaN,70000.000000,60.000000,54.000000,NaN
50%,1054.000000,NaN,NaN,NaN,NaN,NaN,90000.000000,74.000000,73.000000,NaN
75%,1074.000000,NaN,NaN,NaN,NaN,NaN,200000.000000,89.000000,81.000000,NaN


# 🧽 3. Data Cleaning Stage

### 3.1 Handling Missing Values (NaN)
Different columns need different strategies based on meaning and type.

In [ ]:
# 1️⃣ Drop rows where essential fields are missing
df_dropped = df.dropna(subset=['Customer_ID', 'Name'])

# 2️⃣ Impute numeric fields with median
df['Income'] = df['Income'].fillna(df['Income'].median())

# 3️⃣ Fill categorical NaNs with mode
df['Preferred_Channel'] = df['Preferred_Channel'].fillna(df['Preferred_Channel'].mode()[0])

# 4️⃣ Conditional imputation — fill missing Gender based on Name
df.loc[(df['Name'].isin(['Alice','Catherine','Evelyn','Grace','Hannah'])) & (df['Gender'].isna()), 'Gender'] = 'F'
df.loc[(df['Name'].isin(['Bob','David','Frank'])) & (df['Gender'].isna()), 'Gender'] = 'M'

# 5️⃣ Fill missing numeric sequences via interpolation
df['Satisfaction_2022'] = df['Satisfaction_2022'].interpolate()

df.head(10)


#forward and backward filling
# df.bfill()
# df.ffill()

,Customer_ID,Name,Gender,Age,Join_Date,City,Income,Satisfaction_2022,Satisfaction_2023,Preferred_Channel
0,1051,Alice,Female,thirty,15-07-2022,kisumu,90000.0,77,98,Both
1,1092,Grace,Male,40,2020-06-10,NaN,50000.0,96,44,In-store
2,1014,Catherine,M,35,2019/12/15,Mombasa,90000.0,56,81,In-store
3,1071,Frank,M,20,2020-06-10,NAIROBI,70000.0,93,78,Online
4,1060,David,Female,45,2020-06-10,Nairobi,35000.0,57,97,Online
5,1020,Evelyn,F,25,15-07-2022,Eldoret,90000.0,96,80,Online
6,1082,Hannah,F,50,2021-01-05,kisumu,90000.0,84,67,In-store
7,1086,Catherine,Male,25,2020-06-10,kisumu,90000.0,63,46,Online
8,1074,Evelyn,Female,25,2021-01-05,Nairobi,90000.0,66,48,Both
9,1074,Hannah,Female,NaN,15-07-2022,Eldoret,1000000.0,85,47,Online


### 3.2 Removing Duplicates

In [ ]:
print(f"Duplicates before: {df.duplicated().sum()}")
df = df.drop_duplicates()
print(f"Duplicates after: {df.duplicated().sum()}")

Duplicates before: 5
Duplicates after: 0


### 3.3 Fixing Data Types

In [ ]:
# Convert Age to numeric
df['Age'] = pd.to_numeric(df['Age'], errors='coerce')

# Convert Join_Date to datetime with flexible formats
df['Join_Date'] = pd.to_datetime(df['Join_Date'], errors='coerce', dayfirst=True)

df.dtypes

,0
Customer_ID,int64
Name,object
Gender,object
Age,float64
Join_Date,datetime64[ns]
City,object
Income,float64
Satisfaction_2022,int64
Satisfaction_2023,int64
Preferred_Channel,object


### 3.4 String Normalization and Categorical Cleanup

In [ ]:
# Strip spaces, normalize case
df['City'] = df['City'].str.strip().str.title()

# Normalize Gender labels
df['Gender'] = df['Gender'].replace({'male':'M','female':'F','Male':'M','Female':'F'})

# Ensure categorical columns have consistent categories
df['Preferred_Channel'] = df['Preferred_Channel'].replace({'both':'Both','online':'Online','in-store':'In-store'})

df['Gender'].value_counts(), df['City'].unique()

(Gender
 F    30
 M    30
 Name: count, dtype: int64,
 array(['Kisumu', nan, 'Mombasa', 'Nairobi', 'Eldoret'], dtype=object))

### 3.5 Handling Outliers (IQR & Capping)

In [ ]:
Q1, Q3 = df['Income'].quantile([0.25, 0.75])
IQR = Q3 - Q1
lower, upper = Q1 - 1.5*IQR, Q3 + 1.5*IQR

df['Income_Capped'] = np.where(df['Income'] > upper, upper, np.where(df['Income'] < lower, lower, df['Income']))

df[['Income', 'Income_Capped']].describe()

,Income,Income_Capped
count,60.000000,60.000000
mean,221666.666667,141000.000000
std,312397.327766,113603.533008
min,35000.000000,35000.000000
25%,70000.000000,70000.000000
50%,90000.000000,90000.000000
75%,200000.000000,200000.000000
max,1000000.000000,395000.000000


In [ ]:
df[['Income', 'Income_Capped']]

,Income,Income_Capped
0,90000.0,90000.0
1,50000.0,50000.0
2,90000.0,90000.0
3,70000.0,70000.0
4,35000.0,35000.0
5,90000.0,90000.0
6,90000.0,90000.0
7,90000.0,90000.0
8,90000.0,90000.0
9,1000000.0,395000.0


### 3.6 Creating New Features

In [ ]:
# Create a new derived feature
df['Years_Since_Join'] = 2025 - df['Join_Date'].dt.year
df['Income_per_Age'] = df['Income_Capped'] / df['Age']

df[['Age','Join_Date','Years_Since_Join','Income_per_Age']].head()

,Age,Join_Date,Years_Since_Join,Income_per_Age
0,NaN,2022-07-15,3.0,NaN
1,40.0,NaT,NaN,1250.000000
2,35.0,NaT,NaN,2571.428571
3,20.0,NaT,NaN,3500.000000
4,45.0,NaT,NaN,777.777778


# 🔁 4. Data Transformation Stage

### 4.1 Column Renaming and Reordering

In [ ]:
df = df.rename(columns={'Income_Capped':'Annual_Income','Preferred_Channel':'Channel'})
cols = ['Customer_ID','Name','Gender','Age','City','Annual_Income','Satisfaction_2022','Satisfaction_2023','Channel','Join_Date']
df = df[cols + [c for c in df.columns if c not in cols]]
df.head()

,Customer_ID,Name,Gender,Age,City,Annual_Income,Satisfaction_2022,Satisfaction_2023,Channel,Join_Date,Income,Years_Since_Join,Income_per_Age
0,1051,Alice,F,NaN,Kisumu,90000.0,77,98,Both,2022-07-15,90000.0,3.0,NaN
1,1092,Grace,M,40.0,NaN,50000.0,96,44,In-store,NaT,50000.0,NaN,1250.000000
2,1014,Catherine,M,35.0,Mombasa,90000.0,56,81,In-store,NaT,90000.0,NaN,2571.428571
3,1071,Frank,M,20.0,Nairobi,70000.0,93,78,Online,NaT,70000.0,NaN,3500.000000
4,1060,David,F,45.0,Nairobi,35000.0,57,97,Online,NaT,35000.0,NaN,777.777778


### 4.2 Sorting and Reindexing

In [ ]:
df = df.sort_values(by=['City','Annual_Income'], ascending=[True, False]).reset_index(drop=True)
df.head()

,Customer_ID,Name,Gender,Age,City,Annual_Income,Satisfaction_2022,Satisfaction_2023,Channel,Join_Date,Income,Years_Since_Join,Income_per_Age
0,1074,Hannah,F,NaN,Eldoret,395000.0,85,47,Online,2022-07-15,1000000.0,3.0,NaN
1,1029,David,M,25.0,Eldoret,200000.0,67,74,Both,NaT,200000.0,NaN,8000.0
2,1013,Bob,M,NaN,Eldoret,200000.0,60,78,Both,NaT,200000.0,NaN,NaN
3,1083,Frank,F,40.0,Eldoret,200000.0,82,75,In-store,NaT,200000.0,NaN,5000.0
4,1020,Evelyn,F,25.0,Eldoret,90000.0,96,80,Online,2022-07-15,90000.0,3.0,3600.0


### 4.3 Reshaping: Wide to Long (Melt)

In [ ]:
df_long = pd.melt(df,
    id_vars=['Customer_ID','Name','Gender','City'],
    value_vars=['Satisfaction_2022','Satisfaction_2023'],
    var_name='Year', value_name='Satisfaction_Score'
)
df_long.head(10)

,Customer_ID,Name,Gender,City,Year,Satisfaction_Score
0,1074,Hannah,F,Eldoret,Satisfaction_2022,85
1,1029,David,M,Eldoret,Satisfaction_2022,67
2,1013,Bob,M,Eldoret,Satisfaction_2022,60
3,1083,Frank,F,Eldoret,Satisfaction_2022,82
4,1020,Evelyn,F,Eldoret,Satisfaction_2022,96
5,1057,Evelyn,M,Eldoret,Satisfaction_2022,97
6,1038,Grace,F,Eldoret,Satisfaction_2022,73
7,1046,Alice,M,Eldoret,Satisfaction_2022,74
8,1002,Grace,F,Eldoret,Satisfaction_2022,94
9,1003,David,M,Eldoret,Satisfaction_2022,93


### 4.4 Reshaping: Long to Wide (Pivot)

In [ ]:
df_long['Year'] = df_long['Year'].str.extract('(\\d{4})')
df_pivot = df_long.pivot_table(index=['Customer_ID','Name'], columns='Year', values='Satisfaction_Score').reset_index()
df_pivot.head()

Year,Customer_ID,Name,2022,2023
0,1001,Alice,93.0,79.0
1,1001,David,53.0,63.0
2,1001,Hannah,84.0,68.0
3,1002,Bob,51.0,87.0
4,1002,Grace,94.0,90.0


### 4.5 Aggregation and Grouping

In [ ]:
city_summary = df.groupby('City')[['Annual_Income','Satisfaction_2023']].agg(['mean','median','count']).reset_index()
city_summary.head()

City  Annual_Income                Satisfaction_2023             
                     mean   median count              mean median count
0  Eldoret  143500.000000  90000.0    10         71.400000   74.5    10
1   Kisumu  138571.428571  90000.0     7         67.857143   67.0     7
2  Mombasa  130454.545455  90000.0    11         79.545455   78.0    11
3  Nairobi  139800.000000  90000.0    25         66.120000   66.0    25

### 4.6 Merging and Concatenation

In [ ]:
# Split and merge example
df_a = df.iloc[:30]
df_b = df.iloc[30:]

merged_df = pd.concat([df_a, df_b])
merged_df.shape

(60, 13)

# ✅ 5. Final Clean Data Overview

In [ ]:
print('Final shape:', df.shape)
print('Missing values per column:')
print(df.isna().sum())
df.head(10)

Final shape: (60, 13)
Missing values per column:
Customer_ID           0
Name                  0
Gender                0
Age                  12
City                  7
Annual_Income         0
Satisfaction_2022     0
Satisfaction_2023     0
Channel               0
Join_Date            47
Income                0
Years_Since_Join     47
Income_per_Age       12
dtype: int64


,Customer_ID,Name,Gender,Age,City,Annual_Income,Satisfaction_2022,Satisfaction_2023,Channel,Join_Date,Income,Years_Since_Join,Income_per_Age
0,1074,Hannah,F,NaN,Eldoret,395000.0,85,47,Online,2022-07-15,1000000.0,3.0,NaN
1,1029,David,M,25.0,Eldoret,200000.0,67,74,Both,NaT,200000.0,NaN,8000.000000
2,1013,Bob,M,NaN,Eldoret,200000.0,60,78,Both,NaT,200000.0,NaN,NaN
3,1083,Frank,F,40.0,Eldoret,200000.0,82,75,In-store,NaT,200000.0,NaN,5000.000000
4,1020,Evelyn,F,25.0,Eldoret,90000.0,96,80,Online,2022-07-15,90000.0,3.0,3600.000000
5,1057,Evelyn,M,50.0,Eldoret,90000.0,97,76,In-store,2022-07-15,90000.0,3.0,1800.000000
6,1038,Grace,F,25.0,Eldoret,90000.0,73,71,Both,NaT,90000.0,NaN,3600.000000
7,1046,Alice,M,35.0,Eldoret,70000.0,74,54,Online,NaT,70000.0,NaN,2000.000000
8,1002,Grace,F,20.0,Eldoret,50000.0,94,90,Online,NaT,50000.0,NaN,2500.000000
9,1003,David,M,45.0,Eldoret,50000.0,93,69,Online,NaT,50000.0,NaN,1111.111111


---
## 🎯 Key Takeaways
- Always **profile your data** before cleaning.
- Choose imputation strategy **based on meaning**, not just statistics.
- Normalize and validate text/categorical data.
- Use **reshaping** (melt/pivot) to prepare for visualization or modeling.
- Create **derived features** for deeper insights.
- Keep transformations **documented and reproducible**.